<a href="https://colab.research.google.com/github/EugeneHsiung/sqlite_database_operations/blob/main/sqlite_database_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading in Packages

In [29]:
import pandas as pd
import re
import sqlite3
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

# 1. Data Exploration and Analysis:
- Import pricing transparency datasets from at least two different hospital systems into a Pandas DataFrame.
-Conduct a basic exploratory analysis using Python, focusing on aspects such as data distribution, missing values, and basic statistics.
-ocument any captivating insights or observations derived from your analysis.
-My expection is to have at least basic descriptive statistics for numerical columns, and frequency counts for categorical columns

In [30]:
nyp = pd.read_json('https://raw.githubusercontent.com/hantswilliams/HHA_504_2023/main/WK3/data/nyp/133957095_NewYorkPresbyterianHospital_standardcharges.json')
sbu = pd.read_csv('https://raw.githubusercontent.com/hantswilliams/HHA_504_2023/main/WK3/data/stonybrook/stonybrook.csv')

In [31]:
nyp

,Code (CPT/DRG),Description,Rev Code,Inpatient/Outpatient,Gross Charges,Discounted Cash Price,Aetna,Cigna,Empire Blue Cross Blue Shield,Emblem Health,...,Consumer Health Network,Devon,Equian,First Health,Magnacare,Multiplan/Beechstreet/PHCS,QHM,Worldwide,Minimum Negotiated Charge,Maximum Negotiated Charge
0,96360,HC IV INFUSION HYDRATION INITIAL 31 MIN-1HR,0260,Inpatient/Outpatient,866.000000,866.000000,641.706,494.486,640.84,1388.424,...,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,253.048250,1388.424000
1,96361,HC IV INFUSION HYDRATION FLUIDS ADDL HR,0260,Inpatient/Outpatient,358.000000,358.000000,265.278,204.418,264.92,1305.112,...,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,51.903250,1990.891341
2,96365,HC IV INFUSION FOR THER/PROPH/DIAG INITIAL UP ...,0260,Inpatient/Outpatient,1397.000000,1397.000000,1035.177,797.687,1033.78,1475.508,...,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,253.048250,1475.508000
3,96367,HC IV INF THER/PROPH/DIAG ADDL SEQ NEW DRUG UP...,0260,Inpatient/Outpatient,480.000000,480.000000,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,82.650750,1325.120000
4,96368,HC IV INFUSION FOR THER/PROPH/DIAG CONCURRENT,0260,Inpatient/Outpatient,480.000000,480.000000,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,156.982800,1325.120000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Multiple,Inpatient,184095.129020,184095.129020,86723.5232,98911.694706,82299.84,115129.89148,...,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,12561.144027,115129.891480
6189,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Multiple,Inpatient,85385.941857,85385.941857,57748.8352,65864.888166,63931.455093,76664.51828,...,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,7833.746854,76664.518280
6190,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Multiple,Inpatient,411665.999955,411665.999955,114736.9984,130862.199072,214294.876425,152319.20576,...,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,21016.798830,214294.876425
6191,988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Multiple,Inpatient,126522.871140,126522.871140,58654.7264,66898.093812,91907.6928,77867.13496,...,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,10429.428059,91907.692800


In [32]:
sbu

,Code,Description,Type,Package/Line_Level,Gross charge,Discounted cash price,De-identified min contracted rate,De-identified max contracted rate,Derived contracted rate,1199-Commercial other,...,Optum-Commercial other,Oxford-Commercial other,Oxford-Commercial HMO/POS,Tricare-Commercial other,United Healthcare-Commercial other,United Healthcare-Medicare Advantage HMO,United Healthcare-Commercial HMO/POS,United Healthcare-Medicaid HMO,United Healthcare-Commercial PPO/Open Access,Veteran Family-Commercial other
0,10004,"FINE NEEDLE ASPIRATION BIOPSY, EACH ADDITIONAL...",Outpatient,Line,718.67,718.67,2155.50,2155.50,2155.50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10005,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Line,2061.25,2061.25,87.02,4782.00,2308.44,2411.0,...,NaN,4782.0,NaN,NaN,3845.25,NaN,NaN,658.94,NaN,NaN
2,10005,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Package,2061.25,2061.25,87.02,4782.00,2308.44,2411.0,...,NaN,4782.0,NaN,NaN,3845.25,NaN,NaN,658.94,NaN,NaN
3,10006,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Line,728.54,728.54,58.55,1761.25,909.90,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10009,FINE NEEDLE ASPIRATION BIOPSY OF GROWTH USING ...,Outpatient,Line,2823.01,2823.01,446.28,6017.59,4219.31,4592.0,...,NaN,NaN,NaN,NaN,5127.00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705,30981203,R&B SICU 09W1 ICR PT-Z,NaN,NaN,12090.00,12090.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7706,30993018,R&B TICR 09W2 - PT-I,NaN,NaN,13590.00,13590.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7707,33500000,R&B REV PICU 11S2 PT-X,NaN,NaN,8165.00,8165.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7708,30333058,ROPH HOSP.OBSERVATION PT-T,NaN,NaN,8165.00,8165.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
nyp.describe()     #statistics for numerical columns

,Gross Charges,Discounted Cash Price,Minimum Negotiated Charge,Maximum Negotiated Charge
count,6.193000e+03,6.193000e+03,5912.000000,5.912000e+03
mean,3.385988e+04,3.385988e+04,2249.918880,2.390277e+04
std,1.062409e+05,1.062409e+05,6015.717204,7.136503e+04
min,1.000000e+00,1.000000e+00,0.454400,1.608000e+00
25%,5.970000e+02,5.970000e+02,122.228437,6.380010e+02
50%,3.925000e+03,3.925000e+03,431.142905,4.086731e+03
75%,3.033600e+04,3.033600e+04,2107.000000,1.925640e+04
max,2.538980e+06,2.538980e+06,204861.126000,1.713460e+06


In [72]:
print(nyp.columns)

Index(['code__cpt_drg_', 'description', 'rev_code', 'inpatient_outpatient',
       'gross_charges', 'discounted_cash_price', 'aetna', 'cigna',
       'empire_blue_cross_blue_shield', 'emblem_health', 'united_health_group',
       'aetna_medicare', 'agewell_medicare', 'emblem_medicare',
       'empire_medicare', 'fidelis_medicare', 'healthfirst_medicare',
       'uhc_community_plan_united_medicare', 'vns_medicare',
       'wellcare_medicare', '1199', 'affinity_molina_essential',
       'affinity_molina_medicaid_chp', 'amida_care_medicaid',
       'emblem_medicaid_chp', 'empire_healthplus_essential',
       'empire_healthplus_exchange', 'empire_healthplus_medicaid_chp',
       'fidelis_essential_exchange', 'fidelis_medicaid_chp',
       'healthfirst_essential_exchange', 'healthfirst_medicaid_chp',
       'mvp_medicaid_chp', 'mvp_essential', 'united_community_plan_essential',
       'united_community_plan_medicaid', 'vns_medicaid',
       'consumer_health_network', 'devon', 'equian', 'fir

In [78]:
# Frequency counts of each of the unique 'gross_charges' in nyp db
nyp['gross_charges'].value_counts()

122.000000      32
3337.000000     29
2553.000000     27
618.000000      26
116.000000      23
                ..
47682.000000     1
26559.000000     1
39648.000000     1
49748.000000     1
77081.639496     1
Name: gross_charges, Length: 4101, dtype: int64

In [34]:
sbu.describe()       #statistics for numerical columns

,Gross charge,Discounted cash price,De-identified min contracted rate,De-identified max contracted rate,Derived contracted rate,1199-Commercial other,Aetna-Medicare Advantage HMO,Aetna-Commercial HMO/POS,Aetna-Commercial PPO/Open Access,Aetna-Commercial other,...,Optum-Commercial other,Oxford-Commercial other,Oxford-Commercial HMO/POS,Tricare-Commercial other,United Healthcare-Commercial other,United Healthcare-Medicare Advantage HMO,United Healthcare-Commercial HMO/POS,United Healthcare-Medicaid HMO,United Healthcare-Commercial PPO/Open Access,Veteran Family-Commercial other
count,7.710000e+03,7.710000e+03,6254.000000,6.254000e+03,6254.000000,1014.000000,1106.000000,1827.000000,1612.000000,128.000000,...,943.000000,1777.000000,0.0,434.000000,2357.000000,624.000000,133.000000,2065.000000,357.000000,46.000000
mean,2.274832e+04,2.274832e+04,4090.693876,1.003579e+04,6315.587725,5791.542456,5614.005027,4320.418248,4436.188381,1703.756250,...,10868.205705,2572.653849,NaN,2070.212120,5977.464298,4438.776234,3261.594586,2454.314228,789.416667,702.261087
std,7.470570e+04,7.470570e+04,10576.213503,2.915075e+04,14601.846406,24302.352403,12671.487487,8718.537182,8881.733882,5124.232802,...,20703.429246,4036.667401,NaN,8501.228677,16227.981498,9982.579323,10266.478193,11009.140577,2880.486830,1717.006016
min,3.000000e-02,3.000000e-02,0.010000,1.000000e-01,0.030000,0.010000,1.100000,0.020000,0.500000,0.590000,...,2.370000,0.010000,NaN,1.000000,0.010000,1.330000,0.190000,0.010000,0.180000,2.480000
25%,1.065895e+03,1.065895e+03,96.520000,6.982500e+02,403.117500,148.797500,125.070000,277.900000,312.750000,110.387500,...,200.450000,197.880000,NaN,71.220000,258.500000,96.290000,129.000000,42.980000,8.400000,26.810000
50%,4.745295e+03,4.745295e+03,1108.860000,4.311000e+03,2600.740000,974.400000,714.860000,2153.000000,2153.000000,404.500000,...,1700.200000,1932.380000,NaN,236.230000,1571.000000,387.500000,444.000000,425.790000,34.830000,82.710000
75%,1.103881e+04,1.103881e+04,4309.532500,8.170000e+03,5870.540000,4592.000000,5782.430000,4822.000000,4822.000000,1778.920000,...,13359.895000,4704.000000,NaN,1486.947500,5127.000000,3531.920000,2257.430000,1753.100000,286.800000,235.000000
max,1.857952e+06,1.857952e+06,327869.690000,1.302636e+06,370099.900000,331304.280000,157848.600000,99999.990000,96372.570000,54838.130000,...,277267.380000,80891.260000,NaN,154940.610000,291490.750000,92493.910000,88051.340000,327869.690000,26350.000000,7195.400000


In [74]:
print(sbu.columns)

Index(['Code', 'Description', 'Type', 'Package/Line_Level', 'Gross charge',
       'Discounted cash price', 'De-identified min contracted rate',
       'De-identified max contracted rate', 'Derived contracted rate',
       '1199-Commercial other', 'Aetna-Medicare Advantage HMO',
       'Aetna-Commercial HMO/POS', 'Aetna-Commercial PPO/Open Access',
       'Aetna-Commercial other', 'Empire Health-Commercial other',
       'Empire Health-Commercial PPO/Open Access',
       'BlueCross BlueShield-Commercial other',
       'Beacon Health-Commercial other', 'Carelon Health-Commercial other',
       'Cigna-Commercial PPO/Open Access', 'Cigna-Commercial other',
       'Cigna-Commercial HMO/POS', 'EH Facet-Commercial other',
       'EmblemHealth-Commercial PPO/Open Access',
       'EmblemHealth-Commercial other', 'EmblemHealth-Commercial HMO/POS',
       'EmblemHealth-Medicaid HMO', 'EmblemHealth-Medicare Advantage HMO',
       'Empire Health-Commercial HMO/POS',
       'Empire Health-Medicare 

In [77]:
# Frequency counts of each of the unique 'Gross charge' in sbu db
sbu['Gross charge'].value_counts()

8165.00     268
12090.00     70
13590.00     63
8965.00      47
513.00       13
           ... 
6453.95       1
3727.11       1
2284.63       1
18455.69      1
4580.00       1
Name: Gross charge, Length: 5871, dtype: int64

In [35]:
# Check for missing values
nyp.isnull().sum()

Code (CPT/DRG)                          0
Description                             0
Rev Code                                0
Inpatient/Outpatient                    0
Gross Charges                           0
Discounted Cash Price                   0
Aetna                                   0
Cigna                                   0
Empire Blue Cross Blue Shield           0
Emblem Health                           0
United Health Group                     0
Aetna Medicare                          0
AgeWell Medicare                        0
Emblem Medicare                         0
Empire Medicare                         0
Fidelis Medicare                        0
Healthfirst Medicare                    0
UHC Community Plan/United Medicare      0
VNS Medicare                            0
WellCare Medicare                       0
1199                                    0
Affinity Molina Essential               0
Affinity Molina Medicaid/CHP            0
Amida Care Medicaid               

In [36]:
# Check for missing values
sbu.isnull().sum()

Code                                               0
Description                                        0
Type                                             783
Package/Line_Level                               783
Gross charge                                       0
Discounted cash price                              0
De-identified min contracted rate               1456
De-identified max contracted rate               1456
Derived contracted rate                         1456
1199-Commercial other                           6696
Aetna-Medicare Advantage HMO                    6604
Aetna-Commercial HMO/POS                        5883
Aetna-Commercial PPO/Open Access                6098
Aetna-Commercial other                          7582
Empire Health-Commercial other                  4650
Empire Health-Commercial PPO/Open Access        5308
BlueCross BlueShield-Commercial other           6456
Beacon Health-Commercial other                  7707
Carelon Health-Commercial other               

In [37]:
nyp.columns = nyp.columns.str.strip()  # Remove leading and trailing white space from a column

In [60]:
# Function to remove white space and special characters from a value
def clean_column_names(df):
    # Define a helper function to clean column names
    def clean_name(name):
        cleaned_name = re.sub(r'[^a-zA-Z0-9]', '_', name)
        return cleaned_name.lower()

    # Rename columns using the helper function
    # This is using a list comprehension - e.g., we have a list to the right of the equals sign,
    # and inside the list, we are applying our function, for every col (or X) that exists in df.columns
    df.columns = [clean_name(col) for col in df.columns]
    return df

# Apply the clean_column_names function to all columns
nyp_clean = clean_column_names(nyp_clean)

In [56]:
sbu.columns = sbu.columns.str.strip()  # Remove leading and trailing white space from a column

In [65]:
# Function to remove white space and special characters from a value
def clean_column_names(df):
    # Define a helper function to clean column names
    def clean_name(name):
        cleaned_name = re.sub(r'[^a-zA-Z0-9]', '_', name)
        return cleaned_name.lower()

    # Rename columns using the helper function
    # This is using a list comprehend - e.g., we have a list to the right of the equals sign,
    # and inside the list, we are applying our function, for every col (or X) that exists in df.columns
    df.columns = [clean_name(col) for col in df.columns]
    return df

# Apply the clean_value function to all columns
sbu_clean = pd.DataFrame()
sbu_clean = clean_column_names(sbu_clean)

# SQLite Database Operations:
- Create a local SQLite database called health.db

- Manual table creation: Manually create a table (schema) using CREATE TABLE SQL query. The table should contain at least 5 columns. Please have at least two columns be numerical columns, and two columns should be categorical (string) columns. Then write at least one INSERT INTO SQL query to populate 1 or 2 rows worth of fake data.

- Automatic table creation: Implement the to_sql function from Pandas to take the example data from Part 1 of this assignment into the SQLite database.

In [44]:
conn = sqlite3.connect('health.db')     #conn is connect, creates file named health db on temporary system
c = conn.cursor()

In [45]:
#create table
c.execute('''
    CREATE TABLE IF NOT EXISTS data (
        patient_name TEXT,
        gender TEXT,
        diagnosis TEXT,
        price REAL
    )
''')

conn.commit()

In [46]:
c.execute('''
  SELECT name
  FROM sqlite_master
  WHERE type='table';
  ''')

c.fetchall()

#for value in c.fetchall():
#    print(value)

[('data',), ('nyp',)]

In [47]:
c.execute('''
  SELECT * FROM data;
''')

print(c.fetchall())

[('Jacob Smith', 'Male', 'Heart Failure', 1000.0), ('Jasmine Smith', 'Female', 'Lung Cancer', 2000.0)]


In [48]:
#inserting data
c.execute('''
    INSERT INTO data (patient_name, gender, diagnosis, price)
    VALUES
    ('Jacob Smith', 'Male', 'Heart Failure', 1000),
    ('Jasmine Smith', 'Female', 'Lung Cancer', 2000)
''')

conn.commit()

In [49]:
c.execute('''
  SELECT * FROM data;
''')

print(c.fetchall())

[('Jacob Smith', 'Male', 'Heart Failure', 1000.0), ('Jasmine Smith', 'Female', 'Lung Cancer', 2000.0), ('Jacob Smith', 'Male', 'Heart Failure', 1000.0), ('Jasmine Smith', 'Female', 'Lung Cancer', 2000.0)]


In [50]:
### create engine to connect to our sqlite DB
engine = create_engine('sqlite:///health.db')

In [51]:
data = pd.read_sql("select * from data;", conn)
data

,patient_name,gender,diagnosis,price
0,Jacob Smith,Male,Heart Failure,1000.0
1,Jasmine Smith,Female,Lung Cancer,2000.0
2,Jacob Smith,Male,Heart Failure,1000.0
3,Jasmine Smith,Female,Lung Cancer,2000.0


In [52]:
import pandas as pd
import sqlite3

In [53]:
df = pd.read_json('https://raw.githubusercontent.com/hantswilliams/HHA_504_2023/main/WK3/data/nyp/133957095_NewYorkPresbyterianHospital_standardcharges.json')
conn = sqlite3.connect('health.db')     # connect to the SQLite database

In [54]:
df.to_sql('nyp', conn, if_exists='replace')

6193

In [55]:
query = """
  SELECT *
  FROM nyp
  WHERE "code" = '0260'
  LIMIT 5;

"""

response = pd.read_sql(query, conn)     # create a new df: response
response

,index,Code (CPT/DRG),Description,Rev Code,Inpatient/Outpatient,Gross Charges,Discounted Cash Price,Aetna,Cigna,Empire Blue Cross Blue Shield,...,Consumer Health Network,Devon,Equian,First Health,Magnacare,Multiplan/Beechstreet/PHCS,QHM,Worldwide,Minimum Negotiated Charge,Maximum Negotiated Charge
